<a href="https://colab.research.google.com/github/CoreTheGreat/HBPU-Machine-Learning-Course/blob/main/ML_Chapter4_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第四章：聚类
湖北理工学院《机器学习》课程资料

作者：李辉楚吴

笔记内容概述:
* 4.1 K均值聚类 k-Means
* 4.2 密度聚类 DBSCAN
* 4.3 电影评分分析中的聚类问题及应用


## 4.1 K均值聚类 k-Means

本章需下载的数据：

* 图片([下载](https://github.com/CoreTheGreat/HBPU-Machine-Learning-Course/tree/main/Data)): car_num.jpg, bmwk.png
* MovieLens电影评分数据[下载](https://files.grouplens.org/datasets/movielens/ml-latest-small.zip)，具体说明见[此处](https://files.grouplens.org/datasets/movielens/ml-latest-small-README.html).

截至2024年8越26日的官方介绍如下（结果可能与课堂讲解有出入）：
These datasets will change over time, and are not appropriate for reporting research results. Small: 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users. Last updated 9/2018.

In [1]:
import os

# Set the environment variable to avoid memory leak warning
os.environ["OMP_NUM_THREADS"] = "2"

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons
from sklearn.datasets import make_blobs

import time

import cv2

import matplotlib.pyplot as plt

color_list = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

label_size = 18 # Label size
ticklabel_size = 14 # Tick label size

### k-Means算法介绍：一维k-Means聚类

In [ ]:
# Generate 1-D data
np.random.seed(42)
x = np.random.rand(100)

# Plot
fig, ax = plt.subplots(figsize=(7,7))
ax.scatter(x, x, edgecolor='black', facecolor='white', linewidth=2, s=7**2)
ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size

# plt.savefig('1D_data_base.png', dpi=300) # Make figure clearer
plt.show()

In [ ]:
# k-Means clustering
k = 2

# Initialize cluster centers
cluster_centers = [0.1, 0.2]

# Initialize distance array
distance = np.zeros((len(x), k))

# Intercation
max_iter = 10
for iter_id in range(max_iter):
    # Calculate distance between points to each center
    for i in range(k):
        distance[:,i] = np.abs(x - cluster_centers[i])

    # Assign to closest centroid
    cluster_idx = np.argmin(distance, axis=1)

    # Display process
    fig, ax = plt.subplots(figsize=(7,7))

    # Display points close to center 1
    ax.scatter(x[cluster_idx == 0], x[cluster_idx == 0], edgecolor=color_list[0], facecolor='white', linewidth=2, s=7**2)
    ax.axvline(x=cluster_centers[0], color=color_list[0], linestyle='--', linewidth=2)

    # Display points close to center 2
    ax.scatter(x[cluster_idx == 1], x[cluster_idx == 1], edgecolor=color_list[1], facecolor='white', linewidth=2, s=7**2)
    ax.axvline(x=cluster_centers[1], color=color_list[1], linestyle='--', linewidth=2)

    ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size
    ax.set_title(f'Interation: {iter_id}, C1: {cluster_centers[0]:.2f}, C2: {cluster_centers[1]:.2f}', fontsize=label_size)

    # plt.savefig(f'1D_data_iter{iter_id}.png', dpi=300) # Make figure clearer
    plt.show()

    # Update cluster centers
    for i in range(k):
        cluster_centers[i] = np.mean(x[cluster_idx == i])

# Display process
fig, ax = plt.subplots(figsize=(7,7))

# Display points close to center 1
ax.scatter(x[cluster_idx == 0], x[cluster_idx == 0], edgecolor=color_list[0], facecolor='white', linewidth=2, s=7**2)
ax.axvline(x=cluster_centers[0], color=color_list[0], linestyle='--', linewidth=2)

# Display points close to center 2
ax.scatter(x[cluster_idx == 1], x[cluster_idx == 1], edgecolor=color_list[1], facecolor='white', linewidth=2, s=7**2)
ax.axvline(x=cluster_centers[1], color=color_list[1], linestyle='--', linewidth=2)

ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size
ax.set_title(f'Interation: {iter_id}, C1: {cluster_centers[0]:.2f}, C2: {cluster_centers[1]:.2f}', fontsize=label_size)

# plt.savefig(f'1D_data_iter{iter_id}.png', dpi=300) # Make figure clearer
plt.show()

### 一维k-Means聚类的应用: 二值分割（Binary Segmentation）

In [ ]:
# Load the car number image
image_carno = cv2.imread('./Data/car_num.jpg')

# Convert from BGR to RGB
image_carno = cv2.cvtColor(image_carno, cv2.COLOR_BGR2RGB)

# Display the image
fig, ax = plt.subplots(figsize=(7,7))
img = ax.imshow(image_carno)
plt.axis('off')  # Hide axes
# plt.savefig('carno_base.png', dpi=300) # Make figure clearer
plt.show()

In [ ]:
# Change image_carno into grey image
image_carno_grey = cv2.cvtColor(image_carno, cv2.COLOR_BGR2GRAY)

# Display the image
fig, ax = plt.subplots(figsize=(7,7))
img = ax.imshow(image_carno_grey, cmap='gray')
plt.axis('off')  # Hide axes
# plt.savefig('carno_grey.png', dpi=300) # Make figure clearer
plt.show()

In [ ]:
# Reshape image_carno_grey into a 1-D vector
x = image_carno_grey.reshape(-1)

# Using k-Means to separate background and foreground by pixels
k = 2

# Initialize cluster centers
cluster_centers = np.random.rand(k) * 255
start_centers = cluster_centers.copy()

# Initialize distance array
distance = np.zeros((len(x), k))

# Intercation
start_time = time.time()
max_iter = 1000
for iter_id in range(max_iter):
    # Calculate distance between points to each center
    for i in range(k):
        distance[:,i] = np.abs(x - cluster_centers[i])

    # Assign to closest centroid
    cluster_idx = np.argmin(distance, axis=1)

    # Update cluster centers
    cluster_centers_prior = cluster_centers.copy()
    for i in range(k):
        cluster_centers[i] = np.mean(x[cluster_idx == i])

    # Check if cluster_centers are stable enough to stop training
    print(f'Iteration {iter_id}: Updated centers {cluster_centers}, Prior centers {cluster_centers_prior}')
    if np.sum(np.abs(cluster_centers-cluster_centers_prior)) == 0:
        break

    cluster_centers_prior = cluster_centers

end_time = time.time()
print(f'Stop after iteration {iter_id}, time consumption is {end_time-start_time}')

# Generate segmentation mask
carno_mask_pixel = np.zeros_like(cluster_idx)
low_value_cluster = np.argmin(cluster_centers)
carno_mask_pixel[cluster_idx != low_value_cluster] = 1 # Set pixels with higher grey value to 1
carno_mask_pixel = carno_mask_pixel.reshape(image_carno_grey.shape)

# Display the mask
fig, ax = plt.subplots(figsize=(7,7))
img = ax.imshow(carno_mask_pixel, cmap='gray')
ax.set_title(f'Final Centers: {cluster_centers}, Iteration: {iter_id}', fontsize=label_size)
plt.axis('off')  # Hide axes
# plt.savefig('carno_mask_pixel_2.png', dpi=300) # Make figure clearer
plt.show()

### 基于k-Means的图像压缩 (Image Compression)


In [ ]:
# Load the car number image
image_bmwk = cv2.imread('./Data/bmwk.png')

# Convert from BGR to RGB
image_bmwk_rgb = cv2.cvtColor(image_bmwk, cv2.COLOR_BGR2RGB)

x_r = image_bmwk_rgb[:, :, 0].reshape(-1) # Store colors in red channel
x_g = image_bmwk_rgb[:, :, 1].reshape(-1) # Store colors in green channel
x_b = image_bmwk_rgb[:, :, 2].reshape(-1) # Store colors in blue channel

# Display the image with no margin
plt.figure(figsize=(image_bmwk_rgb.shape[1]/100, image_bmwk_rgb.shape[0]/100))  # Convert pixels to inches
plt.imshow(image_bmwk_rgb)
plt.axis('off')  # Hide axes
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Remove margins
plt.show()

In [ ]:
def KMeansImage(img, k):
    # Get image size
    w, h, c = img.shape

    # Reshape image along channel
    x = np.reshape(img, (w * h, c))

    # Train k-Means model
    mdl_km = KMeans(n_clusters=k, n_init='auto')
    mdl_km.fit(x)

    # Predict labels of each pixels
    labels = mdl_km.predict(x).reshape(w, h)

    # Get centers
    center_colors = mdl_km.cluster_centers_ / 255.0

    # Use center colors to generate compressed image
    img_comp = np.zeros((w, h, c))
    for i in range(w):
        for j in range(h):
            img_comp[i][j] = center_colors[labels[i][j]]
    return img_comp, center_colors

claster_num = [2, 4, 8, 16, 32, 64]
for k in claster_num:
    img_comp, center_colors = KMeansImage(image_bmwk_rgb, k)

    # Display center colors
    fig, ax = plt.subplots(figsize=(16,1))
    ax.imshow([center_colors])
    plt.axis('off')
    # plt.savefig(f'bmwk_center_{k}.png', dpi=300)
    plt.show()

     # Display compressed image
    plt.figure(figsize=(image_bmwk_rgb.shape[1]/100, image_bmwk_rgb.shape[0]/100))  # Convert pixels to inches
    plt.imshow(img_comp)
    plt.axis('off')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Remove margins
    # plt.savefig(f'bmwk_comp_{k}.png', format='png', bbox_inches='tight', pad_inches=0)
    plt.show()

### 手肘法确定k值

In [ ]:
# Using make_blobs to generate data of ten clustering
X_mb, y_mb = make_blobs(n_samples=500, n_features=2, centers=6, random_state=42)

fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(X_mb[:, 0], X_mb[:, 1], marker="o", c=y_mb, s=10**2, edgecolor="k")
plt.axis('off')
# plt.savefig(f'make_blobs_base.png', dpi=300)
plt.show()

In [ ]:
k_list = np.arange(2, 20, 1)
sse_list = np.zeros(len(k_list))

mdl_km_list = []
for i in range(len(k_list)):
    mdl_km = KMeans(n_clusters=k_list[i], n_init='auto')
    mdl_km.fit(X_mb)
    mdl_km_list.append(mdl_km)
    sse_list[i] = mdl_km.inertia_

# Plot sse_list
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(k_list, sse_list, marker='o', linestyle='-', color='tab:blue')
ax.set_xticks(k_list)
ax.set_xlabel('Number of clusters (k)', fontsize=label_size)
ax.set_ylabel('SSE', fontsize=label_size)
ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size
# plt.savefig(f'make_blobs_sse.png', dpi=300)
plt.show()

In [ ]:
# Display clustering result of k = 5, 6, 7
k_disp = [5, 6, 7]
for k in k_disp:
    mdl_km = mdl_km_list[k-2]

    fig, ax = plt.subplots(figsize=(10,10))
    ax.scatter(X_mb[:, 0], X_mb[:, 1], marker="o", c=mdl_km_list[k-2].labels_, s=10**2, edgecolor="k")
    ax.set_title(f'Number of clusters (k): {k}', fontsize=label_size)
    plt.axis('off')

    # plt.savefig(f'make_blobs_{k}.png', dpi=300)
    plt.show()

## 4.2 DBSCAN 聚类


k-Means聚类的问题，在Moon data上进行聚类

In [ ]:
# Generate virtual data (Moon data)
X_mm, y_mm = make_moons(n_samples=400, noise=0.05, random_state=42)

# Normalize X_mm using z-score
X_mm = (X_mm - np.min(X_mm, axis=0)) / (np.max(X_mm, axis=0)-np.min(X_mm, axis=0))

fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(X_mm[:, 0], X_mm[:, 1], marker="o", c=y_mm, s=10**2, edgecolor="k")
plt.axis('off')
# plt.savefig(f'make_moon_base.png', dpi=300)
plt.show()

用肘部法确定k值


In [ ]:
k_list = np.arange(2, 20, 1)
sse_list = np.zeros(len(k_list))

mdl_km_list = []
for i in range(len(k_list)):
    mdl_km = KMeans(n_clusters=k_list[i], n_init='auto')
    mdl_km.fit(X_mm)
    mdl_km_list.append(mdl_km)
    sse_list[i] = mdl_km.inertia_

# Plot sse_list
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(k_list, sse_list, marker='o', linestyle='-', color='tab:blue')
ax.set_xticks(k_list)
ax.set_xlabel('Number of clusters (k)', fontsize=label_size)
ax.set_ylabel('SSE', fontsize=label_size)
ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size
# plt.savefig(f'make_moon_sse.png', dpi=300)
plt.show()

In [ ]:
# Display clustering result of k = 5, 6, 7
k_disp = [2, 11, 19]
for k in k_disp:
    mdl_km = mdl_km_list[k-2]

    fig, ax = plt.subplots(figsize=(10,10))
    ax.scatter(X_mm[:, 0], X_mm[:, 1], marker="o", c=mdl_km_list[k-2].labels_, s=10**2, edgecolor="k")
    ax.set_title(f'Number of clusters (k): {k}', fontsize=label_size)
    plt.axis('off')

    # plt.savefig(f'make_moon_{k}.png', dpi=300)
    plt.show()

### DBSCAN 聚类基本逻辑

数据分析与参数选择

In [ ]:
min_pts = 3

# Comput Euclidean distance between samples
distance_map = np.zeros((X_mm.shape[0], X_mm.shape[0]))
for i in range(distance_map.shape[0]):
    for j in range(distance_map.shape[1]):
        distance_map[i,j] = np.linalg.norm(X_mm[i] - X_mm[j])

# k-Distance
k_distance = np.sort(np.sort(distance_map, axis=1)[:, min_pts])[::-1]

# Draw k-Distance figure
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(np.arange(1, len(k_distance)+1), k_distance, marker='o', linestyle='-', color='tab:blue')
ax.set_xlabel('Sample Index', fontsize=label_size)
ax.set_ylabel('k-Distance', fontsize=label_size)
ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size
# plt.savefig(f'dbscan_kdistance.png', dpi=300)
plt.show()

确定所有Core Point

In [ ]:
r_eps = 0.04

cluster_labels = np.zeros(X_mm.shape[0]) - 1 # -1 means noise or unlabeled

# Find all core points
snap_noise_id = 0
snap_corepoint_id = 0
for i in range(len(cluster_labels)):
    if np.sum(distance_map[i, :] <= r_eps) >= min_pts:
        cluster_labels[i] = 0 # 0 means unclustered core point

        if snap_corepoint_id < 3:
            fig, ax = plt.subplots(figsize=(10,10))

            # Draw unlabeled or noise point
            noise_idx = (cluster_labels == -1)
            ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

            # Draw eps field of the current core point
            corepoint_idx = (cluster_labels == 0)
            circle = plt.Circle((X_mm[i, 0], X_mm[i, 1]), r_eps, edgecolor='red', facecolor='tab:red', alpha=0.5, zorder=1)
            ax.add_patch(circle)

            # Draw core points
            ax.scatter(X_mm[corepoint_idx, 0], X_mm[corepoint_idx, 1], marker="o", c='red', s=10**2, edgecolor="k", zorder=2)

            ax.set_title(f'Phase 1: Iteration {i}, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
            plt.axis('off')
            # plt.savefig(f'dbscan_corepoint_{snap_corepoint_id+1}.png', dpi=300)
            plt.show()

            snap_corepoint_id += 1
    elif snap_noise_id < 3:
        fig, ax = plt.subplots(figsize=(10,10))

        # Draw unlabeled or noise point
        noise_idx = (cluster_labels == -1)
        ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

        # Draw core points
        corepoint_idx = (cluster_labels == 0)
        ax.scatter(X_mm[corepoint_idx, 0], X_mm[corepoint_idx, 1], marker="o", c='red', s=10**2, edgecolor="k", zorder=2)

        # Draw eps field of the current point
        circle = plt.Circle((X_mm[i, 0], X_mm[i, 1]), r_eps, edgecolor='blue', facecolor='tab:blue', alpha=0.5, zorder=1)
        ax.add_patch(circle)

        ax.set_title(f'Phase 1: Iteration {i}, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
        plt.axis('off')
        # plt.savefig(f'dbscan_noise_{snap_noise_id+1}.png', dpi=300)
        plt.show()

        snap_noise_id += 1

# Cluster all core points
fig, ax = plt.subplots(figsize=(10,10))

# Draw unlabeled or noise point
noise_idx = (cluster_labels == -1)
ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

# Draw core points
corepoint_idx = (cluster_labels == 0)
ax.scatter(X_mm[corepoint_idx, 0], X_mm[corepoint_idx, 1], marker="o", c='red', s=10**2, edgecolor="k", zorder=2)

ax.set_title(f'Phase 1: Final, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
plt.axis('off')
# plt.savefig(f'dbscan_corepoint_final.png', dpi=300)
plt.show()

将互相可达的核心点合并成簇

In [ ]:
# Scan unlabeled core points
corepoint_cluster_labels = cluster_labels.copy()
corepoint_indices = np.where(corepoint_cluster_labels == 0)[0]

snap_corepoint_cluster_itr_idx = 0
snap_corepoint_cluster_idx = 0

cluster_id = 0 # Init cluster label

# Clustering until all core points were successfully asigned labels
while corepoint_cluster_labels[corepoint_indices].min() == 0:

    cluster_id += 1 # Build new cluster

    # Find start point, each cluster will be built completed in each loop
    for start_corepoint_idx in corepoint_indices:
        if corepoint_cluster_labels[start_corepoint_idx] == 0:
            candidate_idx = np.array([start_corepoint_idx]) # Regard the single point as a cluster
            break

    # Repeat until no unlabeled points were found
    while candidate_idx.size > 0:
        if cluster_id == 2 and snap_corepoint_cluster_idx < 1:
            fig, ax = plt.subplots(figsize=(10,10))

            # Draw noise points
            noise_idx = (corepoint_cluster_labels == -1)
            ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

            # Draw unlabeled core points
            unlabel_idx = (corepoint_cluster_labels == 0)
            ax.scatter(X_mm[unlabel_idx, 0], X_mm[unlabel_idx, 1], marker="o", c="r", s=10**2, edgecolor="k", zorder=0)

            # Draw cluster 1 core points
            cluster1_idx = (corepoint_cluster_labels == 1)
            ax.scatter(X_mm[cluster1_idx, 0], X_mm[cluster1_idx, 1], marker="o", c='tab:blue', s=10**2, edgecolor="k", zorder=2)

            # Draw cluster 2 core points
            cluster2_idx = (corepoint_cluster_labels == 2)
            ax.scatter(X_mm[cluster2_idx, 0], X_mm[cluster2_idx, 1], marker="o", c='yellow', s=10**2, edgecolor="k", zorder=2)

            ax.set_title(f'Phase 2: Cluster 1, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
            plt.axis('off')
            # plt.savefig(f'dbscan_corepoint_cluster_1.png', dpi=300)
            plt.show()

            snap_corepoint_cluster_idx += 1

        # Assign cluster label to the first candidate core point
        corepoint_cluster_labels[candidate_idx[0]] = cluster_id
        circle_id = candidate_idx[0]

        # Find the neighbor core points of the first candidate core point
        neighbor_idx = np.where(distance_map[candidate_idx[0]] <= r_eps)[0] # Find neighbor points
        neighbor_idx = neighbor_idx[neighbor_idx != candidate_idx[0]] # Filter out itself
        neighbor_idx = neighbor_idx[corepoint_cluster_labels[neighbor_idx] == 0] # Remain unlabeled core points

        # Concatenate neighbor_idx to candidate_idx
        candidate_idx = np.concatenate((candidate_idx, neighbor_idx))

        # Remove the first candidate_idx core point
        candidate_idx = np.delete(candidate_idx, 0)

        # Remove duplicated points
        candidate_idx = np.unique(candidate_idx)

        if snap_corepoint_cluster_itr_idx < 3:
            fig, ax = plt.subplots(figsize=(10,10))

            # Draw noise points
            noise_idx = (corepoint_cluster_labels == -1)
            ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

            # Draw unlabeled core points
            unlabel_idx = (corepoint_cluster_labels == 0)
            ax.scatter(X_mm[unlabel_idx, 0], X_mm[unlabel_idx, 1], marker="o", c="r", s=10**2, edgecolor="k", zorder=0)

            # Draw eps field of the current core point
            circle = plt.Circle((X_mm[circle_id, 0], X_mm[circle_id, 1]), r_eps, edgecolor='blue', facecolor='tab:blue', alpha=0.5, zorder=1)
            ax.add_patch(circle)

            # Draw cluster 1 core points
            cluster1_idx = (corepoint_cluster_labels == 1)
            ax.scatter(X_mm[cluster1_idx, 0], X_mm[cluster1_idx, 1], marker="o", c='tab:blue', s=10**2, edgecolor="k", zorder=2)

            # Draw cluster 2 core points
            cluster2_idx = (corepoint_cluster_labels == 2)
            ax.scatter(X_mm[cluster2_idx, 0], X_mm[cluster2_idx, 1], marker="o", c='yellow', s=10**2, edgecolor="k", zorder=2)

            ax.set_title(f'Phase 2: Iteration {snap_corepoint_cluster_itr_idx+1}, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
            plt.axis('off')
            # plt.savefig(f'dbscan_corepoint_cluster_Itr{snap_corepoint_cluster_itr_idx}.png', dpi=300)
            plt.show()

            snap_corepoint_cluster_itr_idx += 1

# Display clustering result
fig, ax = plt.subplots(figsize=(10,10))

# Draw noise points
noise_idx = (corepoint_cluster_labels == -1)
ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

# Draw unlabeled core points
unlabel_idx = (corepoint_cluster_labels == 0)
ax.scatter(X_mm[unlabel_idx, 0], X_mm[unlabel_idx, 1], marker="o", c="r", s=10**2, edgecolor="k", zorder=0)

# Draw cluster 1 core points
cluster1_idx = (corepoint_cluster_labels == 1)
ax.scatter(X_mm[cluster1_idx, 0], X_mm[cluster1_idx, 1], marker="o", c='tab:blue', s=10**2, edgecolor="k", zorder=2)

# Draw cluster 2 core points
cluster2_idx = (corepoint_cluster_labels >= 2)
ax.scatter(X_mm[cluster2_idx, 0], X_mm[cluster2_idx, 1], marker="o", c=corepoint_cluster_labels[cluster2_idx], s=10**2, edgecolor="k", zorder=2)

ax.set_title(f'Phase 2: Final, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
plt.axis('off')
# plt.savefig(f'dbscan_corepoint_cluster.png', dpi=300)
plt.show()

将满足条件的边缘点Border Point加入簇中

In [ ]:
borderpoint_labels = corepoint_cluster_labels.copy()

# Get unlabeled points
candidate_indices = np.where(borderpoint_labels == -1)[0]

snap_border_itr = 0

# Scan candidate_idx to find border points
for candidate_idx in candidate_indices:

    # Find the neighbor points of i
    neighbor_idx = np.where(distance_map[candidate_idx] <= r_eps)[0] # Find neighbor points
    neighbor_idx = neighbor_idx[neighbor_idx != candidate_idx] # Filter out itself
    neighbor_idx = neighbor_idx[borderpoint_labels[neighbor_idx] > 0] # Remain core points

    # Candidate point is a border point
    if len(neighbor_idx) > 0:

        # Assign the closest core point's cluster label to the candicate point
        distance_to_corepoint = distance_map[candidate_idx, neighbor_idx]
        closest_corepoint_idx = neighbor_idx[np.argmin(distance_to_corepoint)]
        borderpoint_labels[candidate_idx] = borderpoint_labels[closest_corepoint_idx]

        if snap_border_itr < 3:
            # Display clustering result
            fig, ax = plt.subplots(figsize=(10,10))

            # Draw noise points
            noise_idx = (borderpoint_labels == -1)
            ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

            # Draw eps field of the current core point
            circle = plt.Circle((X_mm[candidate_idx, 0], X_mm[candidate_idx, 1]), r_eps, edgecolor='red', facecolor='tab:red', alpha=0.5, zorder=1)
            ax.add_patch(circle)

            # Draw cluster 1 core points
            cluster1_idx = (borderpoint_labels == 1)
            ax.scatter(X_mm[cluster1_idx, 0], X_mm[cluster1_idx, 1], marker="o", c='tab:blue', s=10**2, edgecolor="k", zorder=2)

            # Draw cluster 2 core points
            cluster2_idx = (borderpoint_labels >= 2)
            ax.scatter(X_mm[cluster2_idx, 0], X_mm[cluster2_idx, 1], marker="o", c=borderpoint_labels[cluster2_idx], s=10**2, edgecolor="k", zorder=2)

            ax.set_title(f'Phase 3: Itr {snap_border_itr}, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
            plt.axis('off')
            # plt.savefig(f'dbscan_border_{snap_border_itr}.png', dpi=300)
            plt.show()

            snap_border_itr += 1

# Display clustering result
fig, ax = plt.subplots(figsize=(10,10))

# Draw noise points
noise_idx = (borderpoint_labels == -1)
ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

# Draw cluster 1 core points
cluster1_idx = (borderpoint_labels == 1)
ax.scatter(X_mm[cluster1_idx, 0], X_mm[cluster1_idx, 1], marker="o", c='tab:blue', s=10**2, edgecolor="k", zorder=2)

# Draw cluster 2 core points
cluster2_idx = (borderpoint_labels >= 2)
ax.scatter(X_mm[cluster2_idx, 0], X_mm[cluster2_idx, 1], marker="o", c=borderpoint_labels[cluster2_idx], s=10**2, edgecolor="k", zorder=2)

ax.set_title(f'Phase 3: Final, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
plt.axis('off')
# plt.savefig(f'dbscan_border_final.png', dpi=300)
plt.show()

人工调整参数后的结果

In [ ]:
mdl_dbscan_eps = 0.05
mdl_dbscan_minpts = 3

# Define DBSCAN model
mdl_dbscan = DBSCAN(eps=mdl_dbscan_eps, min_samples=mdl_dbscan_minpts)

# Train model
start_time = time.time()
mdl_dbscan.fit(X_mm)
end_time = time.time()

print(f'Training time: {end_time - start_time:.2f} seconds')

labels_mdl_dbscan = mdl_dbscan.labels_

# Display clustering result
fig, ax = plt.subplots(figsize=(10,10))

# Draw noise points
noise_idx = (labels_mdl_dbscan == -1)
ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

# Draw cluster 1 core points
cluster1_idx = (labels_mdl_dbscan == 0)
ax.scatter(X_mm[cluster1_idx, 0], X_mm[cluster1_idx, 1], marker="o", c='tab:blue', s=10**2, edgecolor="k", zorder=2)

# Draw cluster 2 core points
cluster2_idx = (labels_mdl_dbscan >= 1)
ax.scatter(X_mm[cluster2_idx, 0], X_mm[cluster2_idx, 1], marker="o", c=labels_mdl_dbscan[cluster2_idx], s=10**2, edgecolor="k", zorder=2)

ax.set_title(f'Adjust parameter manually: eps = {mdl_dbscan_eps}, minPts = {mdl_dbscan_minpts}', fontsize=label_size)
plt.axis('off')
# plt.savefig(f'mdl_dbscan.png', dpi=300)
plt.show()

## 4.3 电影评分分析中的聚类问题及应用

Moves Data File Structure (movies.csv)

Genres: (no genres listed), 动作（Action）, 冒险（Adventure）, 动画（Animation）, 儿童（Children）, 喜剧（Comedy）, 犯罪（Crime）, 纪录片（Documentary）, 剧情（Drama）, 奇幻（Fantasy）, 黑色电影（Film-Noir）, 恐怖（Horror）, IMAX, 音乐（Musical）, 推理（Mystery）, 爱情（Romance）, 科幻（Sci-Fi）, 惊悚（Thriller）, 战争（War）, 西部（Western）

In [ ]:
movies = pd.read_csv('./Data/movies.csv')
print(f'Movie number {len(movies)}')
movies.head(10)

In [ ]:
# Statistic genres
genres_list = []
for genres in movies['genres']:
    genres_list.extend(genres.split('|'))
genres_list = np.sort(np.unique(genres_list))
print(f'Movie genres: {genres_list}')

Rating Data File Structure (ratings.csv)

Each line of this file after the header row represents one rating of one movie by one user, and has the following format:

userId, movieId, rating, timestamp

Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).

Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.

In [ ]:
ratings = pd.read_csv('./Data/ratings.csv')
user_list = np.sort(np.unique(ratings['userId']))
print(f'{len(user_list)} have provided {len(ratings)} rate records')
ratings.head()

统计用户对各种电影类型的喜爱程度，即计算用户对各类电影的平均分

In [5]:
# Creat user-genres array
user_genres_rate = np.zeros((len(user_list), len(genres_list)))
user_genres_rate_counts = np.zeros((len(user_list), len(genres_list)))

for i in range(len(ratings)):
    # User ID start from 1
    # Convert user ID to user_idx by minus 1
    user_idx = int(ratings.iloc[i]['userId'] - 1)
    
    # Get rate
    rate = ratings.iloc[i]['rating']

    # Get target movie
    movie_id = ratings.iloc[i]['movieId']
    
    # Split movie genres string
    genres_type = movies[movies['movieId'] == movie_id]['genres'].values[0]
    genres_type = genres_type.split('|')

    # Statistic rates of movie genres
    for genres in genres_type:
        # Using '[0][0]' to get index from a list tuple
        # First [0] get the index list outputed by np.where()
        # Second [0] get the first item of list (with single item)
        genres_idx = np.where(genres_list == genres)[0][0]

        # Sum rate
        user_genres_rate[user_idx, genres_idx] += rate
        
        # Count movie number of the genres
        user_genres_rate_counts[user_idx, genres_idx] += 1

# Compute genres rates of all users
user_genres_rate = user_genres_rate / user_genres_rate_counts

观察user_genres_rate

In [ ]:
# Display user_genres_rate shape in a table
import pandas as pd

# Convert user_genres_rate to a DataFrame
user_genres_df = pd.DataFrame(user_genres_rate, columns=genres_list)

# Display shape information
print("\nDataFrame Shape:")
print(f"Rows: {user_genres_df.shape[0]}")
print(f"Columns: {user_genres_df.shape[1]}")

# Display the first few rows of the DataFrame, showing only 'Action' and 'Animation'
user_genres_df[['Action', 'Animation']].head()

用散点图展示动作电影（Action）和动画电影（Animation）的用户喜爱情况

可选电影类型: 

'(no genres listed)' 'Action' 'Adventure' 'Animation' 'Children' 'Comedy'

'Crime' 'Documentary' 'Drama' 'Fantasy' 'Film-Noir' 'Horror' 'IMAX'

'Musical' 'Mystery' 'Romance' 'Sci-Fi' 'Thriller' 'War' 'Western'

In [ ]:
genres_1 = 'Action' # Define first genres
genres_2 = 'Animation' # Define second genres

idx_g1 = np.where(genres_list == genres_1)[0][0] # Get genres 1 index in the genres_list
idx_g2 = np.where(genres_list == genres_2)[0][0] # Get genres 2 index in the genres_list

# Get list of user who have watched movies including both genres_1 and genres_2
# Idea: People who have watched related movies are more credible in determining whether they like them or not.
idx_both = np.where(np.logical_and(user_genres_rate[:,idx_g1] > 0, user_genres_rate[:,idx_g2] > 0))[0]
print(f'Number of users who watched both {genres_1} and {genres_2}: {len(idx_both)}')

# Filter rating information of users who have watched both genres_1 and genres_2
x = user_genres_rate[idx_both][:, [idx_g1, idx_g2]]
print(f'Shape of x: {x.shape}')

# Drawing rates distribution by scatter figure
fig, ax = plt.subplots(figsize=(6,6))

ax.scatter(x[:,0], x[:,1], marker="o", c='white', s=10**2, edgecolor="k")

ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size
ax.set_xlabel(f'Average rate of {genres_1}', fontsize=label_size)
ax.set_ylabel(f'Average rate of {genres_2}', fontsize=label_size)
ax.set_xlim([-0.1, 5.1])
ax.set_ylim([-0.1, 5.1])
ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size
# plt.savefig(f'rate_of_interested_movies.png', dpi=300) # Make figure clearer
plt.show()

使用K-Means分类，观察结果，进行分析

In [ ]:
for k in [2, 4, 8]:

    # Define k-means clustering model
    mdl_km = KMeans(n_clusters=k, n_init='auto')

    # Train k-Means model
    mdl_km.fit(x)
    
    # Assign clusters
    labels_km = mdl_km.predict(x)

    # Draw scatter figure
    fig, ax = plt.subplots(figsize=(6,6))

    ax.scatter(x[:,0], x[:,1], marker="o", c=labels_km, s=10**2, edgecolor="k")

    ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size
    ax.set_xlabel(f'Average rate of {genres_1}', fontsize=label_size)
    ax.set_ylabel(f'Average rate of {genres_2}', fontsize=label_size)
    ax.set_xlim([-0.1, 5.1])
    ax.set_ylim([-0.1, 5.1])
    ax.set_title(f'k-Means clustering, k = {k}', fontsize=label_size)
    ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size
    plt.savefig(f'rate_kMeans_{k}.png', dpi=300) # Make figure clearer
    plt.show()

使用DBSCAN分类，观察结果，进行分析

In [ ]:
# Using k-Distance to find potential values of e
min_pts = 3

# Initialize distance_map to store distance
distance_map = np.zeros((len(x), len(x)))

# Compute Manhattan distance using nested loops
for i in range(len(x)):
    for j in range(len(x)):
        # Calculate Manhattan distance between point i and point j
        distance_map[i, j] = np.abs(x[i, 0] - x[j, 0]) + np.abs(x[i, 1] - x[j, 1])

# Get k-Distance from distance_map
k_distance = np.sort(distance_map, axis=1)[:, min_pts]

# Sort k_distance in descend order
k_distance = np.sort(k_distance)[::-1]

# Draw k-Distance figure
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(np.arange(1, len(k_distance)+1), k_distance, marker='o', linestyle='-', color='tab:blue')
ax.set_xlabel('Sample Index', fontsize=label_size)
ax.set_ylabel('k-Distance', fontsize=label_size)
ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size
plt.savefig(f'rate_dbscan_kdistance.png', dpi=300)
plt.show()

In [ ]:
# Take k-Distance around the elbow place
elbow_index = np.argmax(np.diff(np.diff(k_distance)))  # Find the elbow point automatically
k_distance_elbow = k_distance[elbow_index]
print(f"k-Distance at elbow point: {k_distance_elbow:.3f}")

# Define a range of epsilon values around the elbow point
epsilon_range = [0.15, 0.2, 0.25]  # Use the specific values mentioned in the loop below
print("Epsilon values to be used:")
for eps in epsilon_range:
    print(f"  {eps:.3f}")

for e in [0.15, 0.2, 0.25]:

    # Define DBSCAN clustering model
    mdl_dbscan = DBSCAN(eps=e, min_samples=min_pts)

    # Train k-Means model
    start_time = time.time()
    mdl_dbscan.fit(x)
    end_time = time.time()

    print(f'Training time: {end_time - start_time:.2f} seconds')

    labels_dbscan = mdl_dbscan.labels_

    # Plot scatter of average rates
    fig, ax = plt.subplots(figsize=(6,6))

    # Draw noise points
    noise_idx = (labels_dbscan == -1)
    ax.scatter(x[noise_idx,0], x[noise_idx,1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

    # Draw other points
    ax.scatter(x[~noise_idx,0], x[~noise_idx,1], marker="o", c=labels_dbscan[~noise_idx], s=10**2, edgecolor="k")

    ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size
    ax.set_xlabel(f'Average rate of {genres_1}', fontsize=label_size)
    ax.set_ylabel(f'Average rate of {genres_2}', fontsize=label_size)
    ax.set_xlim([-0.1, 5.1])
    ax.set_ylim([-0.1, 5.1])
    ax.set_title(f'DBSCAN clustering, eps = {e}', fontsize=label_size)
    ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size
    plt.savefig(f'rate_dbscan_{e}.png', dpi=300) # Make figure clearer
    plt.show()

进一步分析最大的聚类中的用户倾向：为这些用户推荐电影

In [ ]:
eps = 0.25

# Define DBSCAN clustering model
mdl_dbscan = DBSCAN(eps=eps, min_samples=min_pts)

# Train k-Means model
mdl_dbscan.fit(x)

labels_dbscan = mdl_dbscan.labels_

# Get the biggest cluster from labels_dbscan
# Filter out noise points (labels = -1) before using np.bincount()
biggest_cluster_label = np.argmax(np.bincount(labels_dbscan[labels_dbscan != -1]))
print(f'Biggest cluster label: {biggest_cluster_label}')

# Plot scatter of average rates
fig, ax = plt.subplots(figsize=(6,6))

# Draw noise points
point_idx = (labels_dbscan == biggest_cluster_label)
ax.scatter(x[point_idx,0], x[point_idx,1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size
ax.set_xlabel(f'Average rate of {genres_1}', fontsize=label_size)
ax.set_ylabel(f'Average rate of {genres_2}', fontsize=label_size)
ax.set_xlim([-0.1, 5.1])
ax.set_ylim([-0.1, 5.1])
ax.set_title(f'DBSCAN clustering {biggest_cluster_label}, eps = {eps}', fontsize=label_size)
ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size
plt.savefig(f'rate_dbscan_cluster_{biggest_cluster_label}.png', dpi=300) # Make figure clearer
plt.show()

用热力图的形式展示Cluster 0中用户的打分情况

In [ ]:
# Get rate of users in cluster biggest_cluster_label
user_ratings = ratings[ratings['userId'].isin(idx_both[labels_dbscan == biggest_cluster_label])]
user_movie_ratings = pd.merge(user_ratings, movies, on='movieId')

# Get movie list from user_movie_rating
movie_list = user_movie_ratings['title'].unique()

# Count user number grouped by movie
user_movie_count = user_movie_ratings.groupby('title')['userId'].nunique()

# Find top 20 movies names with the highest counts
top_movies = user_movie_count.sort_values(ascending=False).head(50)
movie_list = top_movies.index.tolist()

# Filter user_movie_ratings
user_top_movie_ratings = user_movie_ratings[user_movie_ratings['title'].isin(movie_list)]

# Using heat map to show movie rate, x-axis is user_id, y-axis is movie
user_movie_ratings_pivot = user_top_movie_ratings.pivot_table(index='userId', columns='title', values='rating', aggfunc='mean') # Change to table
user_movie_ratings_pivot = user_movie_ratings_pivot.fillna(0) # Fill null cell with -1

# Count movie number that each user has rated: rate > 0 means rated
user_movie_count_col = user_movie_ratings_pivot.apply(lambda x: (x > 0).sum(), axis=1)

# Add user_movie_count_col to user_movie_ratings_pivot as the second column
user_movie_ratings_pivot.insert(0, 'user_movie_count', user_movie_count_col)

display(user_movie_ratings_pivot)

In [ ]:
# Total user number
print(f'Total user number: {len(user_movie_ratings_pivot)}')
disp_user_num_left = np.floor(len(user_movie_ratings_pivot) / 2).astype(int)

# Draw head image (left)
fig, ax = plt.subplots(figsize=(100,10))

im = ax.imshow(user_movie_ratings_pivot.iloc[:disp_user_num_left,1:].values.T, cmap='coolwarm') # Draw heat map

# Set x-axis
ax.set_xticks(np.arange(0, disp_user_num_left, 10))
ax.set_xticklabels(np.arange(0, disp_user_num_left, 10))

# Set y-axis
ax.set_yticks(np.arange(len(movie_list)))
ax.set_yticklabels(movie_list, rotation=0)

# Set title and font
ax.set_title('Heat Map of User Movie Ratings', fontsize=label_size)
ax.tick_params(axis='both', which='major', labelsize=ticklabel_size)

plt.savefig(f'rate_heatmap_left.png', dpi=300) # Make figure clearer
plt.show()

# Draw head image (right)
fig, ax = plt.subplots(figsize=(100,10))

im = ax.imshow(user_movie_ratings_pivot.iloc[disp_user_num_left:,1:].values.T, cmap='coolwarm') # Draw heat map

# Set x-axis
ax.set_xticks(np.arange(0, len(user_movie_ratings_pivot)-disp_user_num_left, 10))
ax.set_xticklabels(np.arange(disp_user_num_left, len(user_movie_ratings_pivot), 10))

# Set y-axis
ax.set_yticks(np.arange(len(movie_list)))
ax.set_yticklabels(movie_list, rotation=0)

# Set title and font
ax.set_title('Heat Map of User Movie Ratings', fontsize=label_size)
ax.tick_params(axis='both', which='major', labelsize=ticklabel_size)

plt.savefig(f'rate_heatmap_right.png', dpi=300) # Make figure clearer
plt.show()

按照观看电影数量进行降序排序

In [ ]:
# Sort table
user_movie_ratings_pivot_sort = user_movie_ratings_pivot.sort_values(by='user_movie_count', ascending=False)
user_movie_ratings_pivot_userid = user_movie_ratings_pivot_sort.index

# Total user number
disp_user_num_left = np.floor(len(user_movie_ratings_pivot_sort) / 2).astype(int)

# Draw head image (left)
fig, ax = plt.subplots(figsize=(100,10))

im = ax.imshow(user_movie_ratings_pivot_sort.iloc[:disp_user_num_left,1:].values.T, cmap='coolwarm') # Draw heat map

# Set x-axis
ax.set_xticks(np.arange(0, disp_user_num_left, 10))
ax.set_xticklabels(user_movie_ratings_pivot_userid[np.arange(0, disp_user_num_left, 10)])

# Set y-axis
ax.set_yticks(np.arange(len(movie_list)))
ax.set_yticklabels(movie_list, rotation=0)

# Set title and font
ax.set_title('Heat Map of User Movie Ratings', fontsize=label_size)
ax.tick_params(axis='both', which='major', labelsize=ticklabel_size)

plt.savefig(f'rate_heatmap_sort_left.png', dpi=300) # Make figure clearer
plt.show()

# Draw head image (right)
fig, ax = plt.subplots(figsize=(100,10))

im = ax.imshow(user_movie_ratings_pivot_sort.iloc[disp_user_num_left:,1:].values.T, cmap='coolwarm') # Draw heat map

# Set x-axis
ax.set_xticks(np.arange(0, len(user_movie_ratings_pivot_sort)-disp_user_num_left, 10))
ax.set_xticklabels(np.arange(disp_user_num_left, len(user_movie_ratings_pivot_sort), 10))

# Set y-axis
ax.set_yticks(np.arange(len(movie_list)))
ax.set_yticklabels(movie_list, rotation=0)

# Set title and font
ax.set_title('Heat Map of User Movie Ratings', fontsize=label_size)
ax.tick_params(axis='both', which='major', labelsize=ticklabel_size)

plt.savefig(f'rate_heatmap_sort_right.png', dpi=300) # Make figure clearer
plt.show()

为前用户推荐电影，即对未评分的电影预测用户可能的评分

In [ ]:
# Select the first 100 user from user_movie_ratings_pivot_sort
user_movie_ratings_pivot_sort_100 = user_movie_ratings_pivot_sort.iloc[:100]

user_movie_ratings_pivot_userid = user_movie_ratings_pivot_sort_100.index

# Draw head image (left)
fig, ax = plt.subplots(figsize=(100,10))

im = ax.imshow(user_movie_ratings_pivot_sort_100.iloc[:,1:].values.T, cmap='coolwarm') # Draw heat map

# Set x-axis
ax.set_xticks(np.arange(0, len(user_movie_ratings_pivot_sort_100), 10))
ax.set_xticklabels(user_movie_ratings_pivot_userid[np.arange(0, len(user_movie_ratings_pivot_sort_100), 10)])

# Set y-axis
ax.set_yticks(np.arange(len(movie_list)))
ax.set_yticklabels(movie_list, rotation=0)

# Set title and font
ax.set_title('Heat Map of User Movie Ratings', fontsize=label_size)
ax.tick_params(axis='both', which='major', labelsize=ticklabel_size)

plt.savefig(f'rate_heatmap_sort_top100.png', dpi=300) # Make figure clearer
plt.show()

计算用户相似度(Jaccard)矩阵

In [37]:
# Compute similarity between users
movie_list = list(user_movie_ratings_pivot_sort_100.columns[1:])
user_list = list(user_movie_ratings_pivot_sort_100.index)
sim_matrix = np.zeros((len(user_list), len(user_list)))

for i in range(len(user_list)):
    for j in range(len(user_list)):
        if i == j: # Avoid the influence of itself
            continue
        rate_i = user_movie_ratings_pivot_sort_100.iloc[i,1:]
        rate_j = user_movie_ratings_pivot_sort_100.iloc[j,1:]
        intersection = np.sum((rate_i > 0) & (rate_j > 0))
        union = np.sum((rate_i > 0) | (rate_j > 0))
        sim_matrix[i,j] = intersection / union if union > 0 else 0

# Get user_list_unrate to store users who have movies not been rated
user_list_unrate = {}
for i in range(len(user_movie_ratings_pivot_sort_100)):
    movie_unrate = np.where(user_movie_ratings_pivot_sort_100.iloc[i,1:] == 0)[0] # Start from the first movie while jumping over the first column
    if len(movie_unrate) > 0:
        user_id_unrate = user_movie_ratings_pivot_sort_100.index[i]
        user_list_unrate[user_id_unrate] = np.where(user_movie_ratings_pivot_sort_100.iloc[i,1:] == 0)[0] + 1 # Add 1 to real pos

# Print rate of unrated movies
# for unrate_user_id in user_list_unrate.keys():
#     for unrate_movie_id in user_list_unrate[user_id]:
#         print(user_movie_ratings_pivot_sort_100.iloc[user_list.index(user_id), unrate_movie_id])

对单个例子进行评分

In [ ]:
def rate_movie(unrate_user_id, unrate_movie_id, user_movie_ratings_pivot_sort_100, sim_matrix):
    co_user_num = 10 # Colabarative users - k value

    # User idx who rate unrate_movie_id[i] higher than 0
    unrate_movie_rates = user_movie_ratings_pivot_sort_100.iloc[:,unrate_movie_id]
    co_user_idx = np.where(unrate_movie_rates > 0)[0]

    # Get co-users' rate on the unrated movie
    co_user_rate = unrate_movie_rates.iloc[co_user_idx]

    # Get co-users' similarity to the unrate_user_id
    co_user_sim = sim_matrix[user_list.index(unrate_user_id), co_user_idx]

    # Create co_user_rate_sim by combining co_user_rate with co_user_sim
    co_user_rate_sim = np.array([co_user_rate, co_user_sim]).T

    # Sort co_user_rate_sim
    co_user_rate_sim_sort = co_user_rate_sim[co_user_rate_sim[:,1].argsort()[::-1]]

    # Predict movie rate
    top_rate = co_user_rate_sim_sort[:co_user_num,0]
    top_sim = co_user_rate_sim_sort[:co_user_num,1]
    predict_rate = np.dot(top_rate, top_sim) / np.sum(top_sim)

    return predict_rate
# Single list
unrate_user_id = list(user_list_unrate.keys())[0]
unrate_movie_id = user_list_unrate[unrate_user_id][0]
unrate_movie_name = movie_list[unrate_movie_id]
print(f'User {unrate_user_id} has {unrate_movie_name} ({unrate_movie_id}) unrated movies')

predict_rate = rate_movie(unrate_user_id, unrate_movie_id, user_movie_ratings_pivot_sort_100, sim_matrix)
print(f'Predicted rate of {unrate_movie_name} ({unrate_movie_id}) for user {unrate_user_id} is {predict_rate}')

对所有未评分用户进行电影喜好预测

In [ ]:
user_movie_ratings_pivot_sort_100_fill = user_movie_ratings_pivot_sort_100.copy()

# Predict movie rate of all unrated movies
for unrate_user_id in user_list_unrate.keys():
    for unrate_movie_id in user_list_unrate[unrate_user_id]:
        rate_pred = rate_movie(unrate_user_id, unrate_movie_id, user_movie_ratings_pivot_sort_100, sim_matrix)

        # Fill user_movie_ratings_pivot_sort_100
        user_movie_ratings_pivot_sort_100_fill.iloc[user_list.index(unrate_user_id), unrate_movie_id] = rate_pred

# Draw head image (left)
fig, ax = plt.subplots(figsize=(100,10))

im = ax.imshow(user_movie_ratings_pivot_sort_100_fill.iloc[:,1:].values.T, cmap='coolwarm') # Draw heat map

# Set x-axis
ax.set_xticks(np.arange(0, len(user_movie_ratings_pivot_sort_100_fill), 10))
ax.set_xticklabels(user_movie_ratings_pivot_userid[np.arange(0, len(user_movie_ratings_pivot_sort_100_fill), 10)])

# Set y-axis
ax.set_yticks(np.arange(len(movie_list)))
ax.set_yticklabels(movie_list, rotation=0)

# Set title and font
ax.set_title('Heat Map of User Movie Ratings', fontsize=label_size)
ax.tick_params(axis='both', which='major', labelsize=ticklabel_size)

plt.savefig(f'rate_heatmap_sort_top100_fill.png', dpi=300) # Make figure clearer
plt.show()